In [ ]:
from encodec import EncodecModel
from encodec.utils import convert_audio
import torchaudio
import torch
from IPython.display import Audio

# Instantiate a pretrained EnCodec model
model = EncodecModel.encodec_model_24khz()
model.eval()
model.set_target_bandwidth(1.5)

# Load and pre-process the audio waveform
wav, sr = torchaudio.load("???")
print(f"Samples before resampling for EnCodec: {wav.shape[-1]}")
print(sr, model.sample_rate)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0)
print(f"Samples after resampling for EnCodec: {wav.shape[-1]}")

# Extract discrete codes from EnCodec
with torch.inference_mode():
    encoded_frames = model.encode(wav)
    print(encoded_frames)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]
decoded_wav = model.decode(encoded_frames)
decoded_wav_resampled = torchaudio.functional.resample(decoded_wav, orig_freq=model.sample_rate, new_freq=sr)
print(f"Samples after resampling output of EnCodec: {decoded_wav_resampled.shape[-1]}")

In [ ]:
print(codes.shape)

In [ ]:
Audio(decoded_wav.squeeze().detach(), rate=model.sample_rate)

In [ ]:
# Delete all codes but the first one which is the most important
print(f"{codes.shape=}")
new_codes = codes.clone()
new_codes[:, 1:, :] = torch.zeros(new_codes.shape)[:, 1:, :]
decoded_wav_cut = model.decode([(new_codes, None)])
Audio(decoded_wav_cut.squeeze().detach(), rate=model.sample_rate)